# Headers

In [1]:
import pandas as pd
import time
import numpy as np



# Metrics
from sklearn.metrics import accuracy_score, f1_score

In [2]:
import torch


print("CUDA available: ", torch.cuda.is_available())
print("Number of GPUs: ", torch.cuda.device_count())
print("GPU Name: ", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU available")

device ='cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')


CUDA available:  False
Number of GPUs:  0
GPU Name:  No GPU available
Using cpu device


In [3]:
from utils import train_datapath, test_datapath
from utils import train_val_split

In [4]:
targets_for_test_df = pd.read_csv('data/targets_for_test.csv', index_col=0)


# Data Processing

# TabNet

## Crude


In [5]:
treated_train_df = pd.read_csv('data/treated_train.csv')

X_train, y_train, X_val, y_val = train_val_split(treated_train_df)


In [6]:
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [ ]:
start_time = time.time()


tabnet = TabNetClassifier()

tabnet.fit(
    X_train.values, y_train.values,
    eval_set=[(X_val.values, y_val.values)],
    eval_metric=['accuracy'],
    max_epochs=100, patience=10,
    batch_size=256, virtual_batch_size=128,
    num_workers=0,
    drop_last=False
)


# Save the trained model
tabnet.save_model("models/tabnet_pretrained_model")

end_time = time.time()
minutes = (end_time - start_time) // 60
seconds = (end_time - start_time) % 60
print(f'Time elapsed: {minutes:.0f}m {seconds:.1f}s')
print('--------------------------------------')



d:\01_GitHub\Directional-Forecasting-in-Cryptocurrencies\env_tranformers\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.69048 | val_0_accuracy: 0.5293  |  0:03:57s
epoch 1  | loss: 0.68839 | val_0_accuracy: 0.51614 |  0:07:34s
epoch 2  | loss: 0.68803 | val_0_accuracy: 0.53054 |  0:11:10s
epoch 3  | loss: 0.68773 | val_0_accuracy: 0.53162 |  0:14:44s
epoch 4  | loss: 0.68707 | val_0_accuracy: 0.53107 |  0:18:19s
epoch 5  | loss: 0.689   | val_0_accuracy: 0.52295 |  0:21:54s
epoch 6  | loss: 0.68747 | val_0_accuracy: 0.53099 |  0:25:30s
epoch 7  | loss: 0.68688 | val_0_accuracy: 0.53214 |  0:29:06s
epoch 8  | loss: 0.68653 | val_0_accuracy: 0.53012 |  0:32:40s
epoch 9  | loss: 0.68656 | val_0_accuracy: 0.53176 |  0:36:17s
epoch 10 | loss: 0.68688 | val_0_accuracy: 0.53188 |  0:39:52s
epoch 11 | loss: 0.68624 | val_0_accuracy: 0.5325  |  0:43:26s
epoch 12 | loss: 0.68633 | val_0_accuracy: 0.53158 |  0:47:07s
epoch 13 | loss: 0.6862  | val_0_accuracy: 0.53259 |  0:50:42s
epoch 14 | loss: 0.68604 | val_0_accuracy: 0.53283 |  0:54:17s
epoch 15 | loss: 0.68628 | val_0_accuracy: 0.53251 |  0

d:\01_GitHub\Directional-Forecasting-in-Cryptocurrencies\env_tranformers\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at tabnet_pretrained_model.zip.zip


d:\01_GitHub\Directional-Forecasting-in-Cryptocurrencies\env_tranformers\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


FileNotFoundError: [Errno 2] No such file or directory: 'tabnet_pretrained_model.zip'

In [13]:

# Load the pre-trained model
loaded_model = TabNetClassifier()
loaded_model.load_model("tabnet_pretrained_model.zip")

# Test the loaded model
y_pred = loaded_model.predict(X_val.values)
# Calculate accuracy
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy:.5f}')

# Calculate F1 macro score
f1_macro = f1_score(y_val, y_pred, average='macro')
print(f'Validation F1 Macro Score: {f1_macro:.5f}')


d:\01_GitHub\Directional-Forecasting-in-Cryptocurrencies\env_tranformers\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
d:\01_GitHub\Directional-Forecasting-in-Cryptocurrencies\env_tranformers\Lib\site-packages\pytorch_tabnet\abstract_model.py:454: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`.

Validation Accuracy: 0.53347
Validation F1 Macro Score: 0.39120


# TST 